In [1]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random
from underthesea import word_tokenize
import  nltk.translate.bleu_score as bleu
from sklearn.model_selection import train_test_split
#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
print("Processing the Intents.....")
with open('intents.json',encoding="utf8") as json_data:
    intents = json.load(json_data)
intents

Processing the Intents.....


{'intents': [{'tag': 'loichao',
   'patterns': ['Hello',
    'xin chao',
    'Bạn khỏe không',
    'Có ai ở đó không?',
    'Xin chào',
    'Chúc một ngày tốt lành'],
   'responses': ['Xin chào, cảm ơn bạn đã ghé thăm',
    'Rất vui được gặp lại',
    'Xin chào, tôi có thể giúp gì cho bạn?']},
  {'tag': 'tambiet',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'tạm biệt', 'chào nhé'],
   'responses': ['Hẹn gặp lại, cảm ơn đã ghé thăm',
    'Chúc một ngày tốt lành',
    'Tạm biệt! Hãy quay lại sớm.',
    'Chào cậu. lần khác nói tiếp nhé']},
  {'tag': 'camon',
   'patterns': ['Cảm ơn', 'thanks', 'cảm ơn bạn nha'],
   'responses': ['Vui vẻ giúp đỡ!', 'Không có gì nha', 'Hân hạnh']},
  {'tag': 'Khoe',
   'patterns': ['khoe khong',
    'Bạn có khỏe không',
    'khỏe không?',
    'bạn khỏe không'],
   'responses': ['Tớ vẫn khỏe, chat chit suốt ngày. hi hi']},
  {'tag': 'Thoitiet',
   'patterns': ['thời tiết hôm nay thế nào'],
   'responses': ['Không tệ', 'Trời trong xanh bao la mấy t

In [4]:
#Xây dựng túi từ cho x và y.
words = []
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print('End add to list')

End add to list


In [5]:
f = open('stopwords.txt', 'r', encoding='UTF-8')
data = f.read()
stop_words = data.split('\n')
ignore_words = ['?','ă', 'â', 'ê', 'ô', 'ơ', 'ư', 'ua', 'uô', 'ia', 'yê', 'iê', 'ưa', 'ươ', 'a', 'b', 'c', 'd', 'e', 'f',
                   'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
                   'ph', 'th', 'tr', 'ch', 'nh', 'ng', 'kh', 'gh', 'ngh','(', ')', ',', '.', '/'] + stop_words
words = [w.lower() for w in words if w not in ignore_words]
# Xóa các từ trùng nhau trong words
words = sorted(list(set(words)))

# Xóa các từ trùng nhau trong classes
classes = sorted(list(set(classes)))
# print(ignore_words)
print (len(documents), "documents")
print (len(classes), "classes")
print (len(words), "words")
# print ("documents", documents)
# print ("classes", classes)
# print ("words", words)

791 documents
421 classes
896 words


In [6]:
# Save data x và y
data_x = []
data_y = []
for doc in documents:
    data_x.append(doc[0])
    data_y.append(doc[1])
print("End")


End


In [7]:
(X_train, X_test, y_train, y_test) = train_test_split(data_x, data_y, test_size = .3)

In [8]:
print("Creating the Data for our Model.....")
train_x = []
train_y = []
output = []

print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for i in range(0,len(X_train) - 1):
    bag = []
    # Lấy input(x) để tạo BOW
    pattern_words = X_train[i]
    # Chuyển đổi tất cả sang chữ thường
    pattern_words = [word.lower() for word in pattern_words]
    # Tạo BOW cho x
    for w in words:
        count = pattern_words.count(w)
        bag.append(count) if w in pattern_words else bag.append(0)                
    # Tạo BOW cho y
    output_row = list(output_empty)
    output_row[classes.index(y_train[i])] = 1

    train_x.append(bag)
    train_y.append(output_row)
print("End")

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....
End


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding, Flatten
from tensorflow.keras.optimizers import SGD

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
#fitting and saving the model 
hist = model.fit(np.array(train_x),np.array(train_y), epochs=300, batch_size=8, verbose=1)
# hist = model.fit(np.array(X_train),np.array(y_train), epochs=300, batch_size=5, verbose=1)

Train on 552 samples
Epoch 1/300
552/552 [==============================] - 0s 490us/sample - loss: 6.0485 - acc: 0.0000e+00
Epoch 2/300
552/552 [==============================] - 0s 161us/sample - loss: 6.0216 - acc: 0.0018
Epoch 3/300
552/552 [==============================] - 0s 197us/sample - loss: 5.9994 - acc: 0.0163
Epoch 4/300
552/552 [==============================] - 0s 193us/sample - loss: 5.9663 - acc: 0.0127
Epoch 5/300
552/552 [==============================] - 0s 164us/sample - loss: 5.9146 - acc: 0.0145
Epoch 6/300
552/552 [==============================] - 0s 188us/sample - loss: 5.8439 - acc: 0.0181
Epoch 7/300
552/552 [==============================] - 0s 168us/sample - loss: 5.7571 - acc: 0.0091
Epoch 8/300
552/552 [==============================] - 0s 197us/sample - loss: 5.6483 - acc: 0.0199
Epoch 9/300
552/552 [==============================] - 0s 172us/sample - loss: 5.5435 - acc: 0.0362
Epoch 10/300
552/552 [==============================] - 0s 170us/sample - l

552/552 [==============================] - 0s 143us/sample - loss: 0.1692 - acc: 0.9583
Epoch 83/300
552/552 [==============================] - 0s 147us/sample - loss: 0.1345 - acc: 0.9656
Epoch 84/300
552/552 [==============================] - 0s 143us/sample - loss: 0.1345 - acc: 0.9692
Epoch 85/300
552/552 [==============================] - 0s 126us/sample - loss: 0.1242 - acc: 0.9710
Epoch 86/300
552/552 [==============================] - 0s 132us/sample - loss: 0.1323 - acc: 0.9674
Epoch 87/300
552/552 [==============================] - 0s 148us/sample - loss: 0.1369 - acc: 0.9728
Epoch 88/300
552/552 [==============================] - 0s 126us/sample - loss: 0.1292 - acc: 0.9728
Epoch 89/300
552/552 [==============================] - 0s 130us/sample - loss: 0.1493 - acc: 0.9656
Epoch 90/300
552/552 [==============================] - 0s 128us/sample - loss: 0.1181 - acc: 0.9692
Epoch 91/300
552/552 [==============================] - 0s 127us/sample - loss: 0.1382 - acc: 0.9674
Epo

552/552 [==============================] - 0s 103us/sample - loss: 0.0481 - acc: 0.9873
Epoch 163/300
552/552 [==============================] - 0s 105us/sample - loss: 0.0612 - acc: 0.9855
Epoch 164/300
552/552 [==============================] - 0s 104us/sample - loss: 0.0595 - acc: 0.9855
Epoch 165/300
552/552 [==============================] - 0s 112us/sample - loss: 0.0583 - acc: 0.9891
Epoch 166/300
552/552 [==============================] - 0s 120us/sample - loss: 0.0554 - acc: 0.9855
Epoch 167/300
552/552 [==============================] - 0s 109us/sample - loss: 0.0540 - acc: 0.9801
Epoch 168/300
552/552 [==============================] - 0s 109us/sample - loss: 0.0568 - acc: 0.9873
Epoch 169/300
552/552 [==============================] - 0s 105us/sample - loss: 0.0603 - acc: 0.9855
Epoch 170/300
552/552 [==============================] - 0s 108us/sample - loss: 0.0617 - acc: 0.9837
Epoch 171/300
552/552 [==============================] - 0s 111us/sample - loss: 0.0389 - acc: 0

552/552 [==============================] - 0s 112us/sample - loss: 0.0218 - acc: 0.9946
Epoch 243/300
552/552 [==============================] - 0s 108us/sample - loss: 0.0240 - acc: 0.9946
Epoch 244/300
552/552 [==============================] - 0s 107us/sample - loss: 0.0340 - acc: 0.9873
Epoch 245/300
552/552 [==============================] - 0s 104us/sample - loss: 0.0459 - acc: 0.9891
Epoch 246/300
552/552 [==============================] - 0s 109us/sample - loss: 0.0336 - acc: 0.9909
Epoch 247/300
552/552 [==============================] - 0s 126us/sample - loss: 0.0342 - acc: 0.9928
Epoch 248/300
552/552 [==============================] - 0s 114us/sample - loss: 0.0374 - acc: 0.9873
Epoch 249/300
552/552 [==============================] - 0s 106us/sample - loss: 0.0275 - acc: 0.9928
Epoch 250/300
552/552 [==============================] - 0s 105us/sample - loss: 0.0384 - acc: 0.9855
Epoch 251/300
552/552 [==============================] - 0s 105us/sample - loss: 0.0406 - acc: 0

In [13]:
def clean_up_sentence(sentence):
    #tách từ và chuyển về cùng 1 dạng
    sentence_words = word_tokenize(sentence)
    sentence_words = [word.lower() for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] =  bag[i] + 1
    return bag


def predict_class(sentence, model):
    p = bow(sentence, words)
    results = model.predict(np.array([p]))[0]
  
    ERROR_THRESHOLD = 0.25
    # Lọc các xác xuất lớn hơn 0.25
    # Lưu ý: Thứ tự xác suất dựa vào thứ tự y được truyền vào
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    # sắp xếp các xác xuất theo thứ tự giảm dần
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [14]:
predict_class('Người điều khiển xe mô tô, xe gắn máy trên đường mà trong khí thở có nồng độ cồn vượt quá bao nhiêu thì bị cấm',model)
# predict_class('Xin chào bạn',model)

[{'intent': 'nongdocon_khitho', 'probability': '0.99998724'}]

In [15]:
def getResponse(ints, intents_json):
    # Lấy xác xuất đầu tiên <=> xác xuất cao nhất sau predict
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = np.random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    if(len(ints)>0):
        res = getResponse(ints, intents)
    else:
        return ''
    return res

In [16]:
res = chatbot_response('hello')
print(res)

Xin chào, cảm ơn bạn đã ghé thăm


In [17]:
# Lọc data x và y cho phần đánh giá
len_y_test = len(y_test)
test_X = []
test_y = []
for i in range(0,len_y_test - 1):
    if(y_test[i] in y_train):
        test_X.append(X_test[i])
        test_y.append(y_test[i])

In [18]:
# Lấy references response từ tập test
references = []
res_test = []
for y in test_y:
    found = [x for x in intents['intents'] if x["tag"] == y ][0]
    references_sub = []
    res_test.append(found['responses'])
    for response in found['responses']:
        textSplit = response.split()
        references_sub.append(textSplit)
    references.append(references_sub)
# print(references)
# print(res_test)

In [19]:
# Lấy candidates từ tập test dựa vào model
candidates = []
test_class = []
for x in test_X:
    message = ' '.join(x)
    ints = predict_class(message,model)
    if(len(ints)>0):
        tag = ints[0]['intent']
        test_class.append(tag)
        response = getResponse(ints,intents)
        candidates.append(response.split())
    else:
        candidates.append([])
        test_class.append('')
# print(candidates)

In [20]:
score = bleu.corpus_bleu(references, candidates)
print(score) 

0.6664181277402538


In [21]:
#Đánh giá phân lớp
count = 0 
for j in range(0,len(test_y) -1):
    if(test_class[j] == test_y[j]):
        count = count + 1
print(count/len(test_y))

0.6967741935483871


In [22]:
# test_X
# res_test
# candidates
# Lưu kết quả test vào file json
test_json = []
len_data_test = len(test_y)
for i in range(0,len_data_test - 1):
    item = {
        "question":' '.join(test_X[i]),
        "answer":res_test[i],
        "predict":' '.join(candidates[i])
    }
    test_json.append(item)
jsonString = json.dumps(test_json,ensure_ascii=False, indent=4)
jsonFile = open("result_test_ANN.json", "w",encoding='utf-8')
jsonFile.write(jsonString)
jsonFile.close()

In [23]:
from tkinter import *
def send():
    #The first part, "1.0" means that the input should be read from line one, character zero (ie: the very first character).
    #END is an imported constant which is set to the string "end". The END part means to read until the end of the text box
    #is reached. The only issue with this is that it actually adds a newline to our input. 
    #So, in order to fix it we should change END to end-1c.
    #The -1c deletes 1 character, while -2c would mean delete two characters, and so on.
    msg = EntryBox.get("1.0", "end-1c").strip()
    EntryBox.delete("0.0", END)
    
    if(msg != ''):
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 10 ))
        
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = Tk()
base.title("Hệ thống hỏi đáp pháp luật")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Arial",10,'bold'), text="Gửi", width="12", height=5,
                    bd=0, bg="#fca311", activebackground="#f8edeb",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
 